In [13]:
# 베이즈 분류기 구현 
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

viagra_spam = {'viagra':
    [1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1],
    'spam':
    [1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1]}
df = pd.DataFrame(viagra_spam, columns = ['viagra',
'spam'])
np_data = df.values

In [14]:
# p(H|D) 즉, D라는 데이터가 주어졌을때 H가 발생할 확률 
# H: 특정 사건 혹은 가설 / D: 주어진 데이터 또는 다른 사건 / |: 이 기호는 "주어진"이라는 의미로, 
# D 사건이 주어졌을 때 
# H 사건의 확률을 계산하라는 의미를 내포

sum((np_data[:, 0] == 1) & (np_data[:, 1] == 1)) / 20  # 조건 |교집합| 확률 구현

0.15

In [15]:
p_viagra = sum(np_data[:, 0] == 1) / len(np_data)    # P(D)
p_viagra

0.3

In [16]:
p_spam = sum(np_data[:, 1] == 1) / len(np_data)     # P(H)
p_spam 

0.3

In [17]:
p_v_cap_s = sum((np_data[:, 0] == 1) & (np_data[:, 1] == 1)) / len(np_data)    # P( D |교집합| H )
p_v_cap_s

0.15

In [18]:
p_n_v_cap_s = sum((np_data[:, 0] == 0) & (np_data[:, 1] == 1)) / len(np_data)       # P( ~ D |교집합| H )
p_n_v_cap_s

0.15

In [19]:
p_spam * (p_v_cap_s / p_spam ) / p_viagra   # P(H|D)

0.5

In [20]:
p_spam * (p_n_v_cap_s / p_spam ) / (1-p_viagra)     # p(H|~D) D라는 조건이 없을 때 H가 발생할 확률 

0.2142857142857143

In [ ]:
# 나이브 베이지안 분류기(Naive Bayesian Classifier)의 기본적인 원리를 설명하는 식
# 주어진 입력 특성 X1,X2,...,Xn에 대한 클래스 레이블 Yc의 조건부 확률을 계산하는 방법
# P(Yc): 클래스 Yc의 사전 확률입니다. 이는 모델 학습 데이터에서 클래스  Yc가 관측된 비율로 계산할 수 있다
# ** 수식에서 ∏ i=1n기호는 "곱셈의 기호"로, 특정 수열의 모든 요소를 곱하는 연산을 나타냅니다. 이 기호는 "시그마(Σ)" 기호가 합을 나타내는 것과 유사하게, 여러 항목의 곱을 간결하게 표현하는 데 사용 / product"의 약자로, 곱셈을 의미
# ∏  i=1 n ​P(Xi∣Yc): 모든 특성  Xi Xi가 주어진 클래스  Yc에 대한 조건부 확률의 곱
# 나이브 베이지안 분류기는 이 조건부 확률을 계산할 때 각 특성이 서로 독립적이라고 가정합니다(이것이 'naive'라는 이름의 유래)
# ∏  i=1 n P(Xi): 각 특성 Xi의 무조건적 확률 / 이는 각 특성이 데이터 전체에서 얼마나 자주 나타나는지의 확률을 의미
# ------------------------------------------
# 이 분류 공식은 주어진 특성 X1 ​,X2 ,...,X  n ​에 대해 가능한 모든 클래스 Yc의 조건부 확률을 계산 / 
# 분류 작업을 할 때, 각 클래스에 대해 이 값을 계산하고 가장 확률이 높은 클래스를 선택하여 해당 데이터 포인트의 클래스로 예측


In [27]:
# 나이브 베이지안 분류기 구현 (나이브란 - 모든 특성들이 서로 조건부 독립이라는 간단한(순진한) 가정)
# 1. 데이터 읽기 /Users/mac/Desktop/24년 대학/1학기/머신러닝 프로그래밍/강의코드/gihorse/fraud.csv

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

data_url = "/Users/mac/Desktop/24년 대학/1학기/머신러닝 프로그래밍/강의코드/gihorse/fraud.csv"
df= pd.read_csv(data_url)
df.head()


,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [28]:
# Y 값을 따로 빼내고 X 데이터들을 원핫인코딩으로 처리

del df["ID"] # ID열 제거
Y_data = df.pop("Fraud") # Fraud열은 빼서 Y_data에 저장
Y_data = Y_data.values # 넘파이 array로 바꿈(열 이름X)
x_df = pd.get_dummies(df)   # 이건 이제 원 - 핫 인코딩 처리인거 알아야지 ? - 범주형 데이터를 0,1 로 변환하는거죵 *(불리안 타입 데이터면 T,F로 원 핫 인코딩 됨 )
x_df = x_df.astype(int)    # 그래서 boolean을 정수형으로 변환
x_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
History_arrears,0,0,0,0,1,1,0,1,0,0
History_current,1,0,0,0,0,0,1,0,1,0
History_none,0,0,0,0,0,0,0,0,0,1
History_paid,0,1,1,1,0,0,0,0,0,0
CoApplicant_coapplicant,0,0,0,0,0,0,0,0,0,0
CoApplicant_guarantor,0,0,0,1,0,0,0,0,0,0
CoApplicant_none,1,1,1,0,1,1,1,1,1,1
Accommodation_free,0,0,0,0,0,0,0,0,0,0
Accommodation_own,1,1,1,0,1,1,1,1,0,1
Accommodation_rent,0,0,0,1,0,0,0,0,1,0


In [29]:
x_data = x_df.values # 넘파이 array로 바꿈
x_data

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]])

In [30]:
# Y 값(사기꾼O/X)이 True인 경우와 False인 경우
P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True
P_Y_True,P_Y_False

(0.3, 0.7)

In [31]:
#  특성(열)이 독립적이라고 가정하고, 전체 데이터프레임의 모든 특성이 동시에 나타날 확률을 계산
p_x=np.prod(x_df.sum(axis=0)/len(x_df))
p_x

4.8808593750000006e-08

In [32]:
# P(Y true)와 P(Y false)의 인덱스 값 정리
ix_Y_True = np.where(Y_data) #Y가 true인 경우의 행 번호
ix_Y_False = np.where(Y_data==False) #Y가 false인 경우의 행 번호
ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12]),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19]),))

In [34]:
# P(Xi | Y true)
# P(Xi | Y false)
# ex) P("free"∣Spam)는 스팸 메일에 "free" 단어가 포함될 확률 / 
# P("free"∣Not Spam)P("free"∣Not Spam)는 비스팸 메일에 "free" 단어가 포함될 확률을 각각 나타냄.

p_x_y_true = (x_data[ix_Y_True].sum(axis=0)) / sum(Y_data==True)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data==False)
p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

In [35]:
# 새로운 데이터 들어왔을 때
x_test = [0,1,0,0,0,1,0, 0,1,0]
r_t=P_Y_True * p_x_y_true.dot(x_test) / p_x
r_f=P_Y_False * p_x_y_false.dot(x_test) / p_x
r_t, r_f 

(8195278.111244496, 15366146.45858343)

In [36]:
r_t<r_f

True

In [37]:
# 사이킷런을 활용한 나이브 베이지안 분류기 사용 
# 하나의 문장이 있을 때 이 문장을 sports와 not sports로 나누는 분류기 만들기 ‐ 사이킷런의 클래스를 사용

y_example_text = ["Sports","Not sports","Sports","Sports","Not sports"]
y_example = [1 if c=="Sports" else 0 for c in y_example_text ]
text_example = ["A great game game", "The The election was over", "Very clean game match", "A clean but forgettable game game","It was a close election", ]

In [38]:
from sklearn.feature_extraction.text import CountVectorizer   # 텍스트 데이터를 토큰화하고 각 토큰의 발생 빈도를 카운트하여 BOW(Bag of Words) 모델

countvect_example = CountVectorizer()
X_example = countvect_example.fit_transform(text_example)    # Fit: 데이터에서 각 단어의 어휘를 학습하고, 각 단어에 고유한 정수 인덱스를 부여
                                                            # Transform: 각 문서(또는 텍스트)를 해당 어휘의 벡터로 변환 / 벡터의 각 요소는 특정 단어가 문서에 나타난 횟수(frequency)를 나타냄.
countvect_example.get_feature_names_out()    # 메소드는 벡터화 과정에서 학습된 어휘의 목록을 반환 / 이 목록은 토큰의 배열로, 각 토큰은 행렬에서 열의 인덱스와 대응

array(['but', 'clean', 'close', 'election', 'forgettable', 'game',
       'great', 'it', 'match', 'over', 'the', 'very', 'was'], dtype=object)

In [39]:
# 이미 훈련된 CountVectorizer 모델(countvect_example)을 사용하여 제공된 text_example 데이터를 벡터화 /
# 이 메소드는 학습된 어휘 사전을 기반으로 각 문서의 단어 빈도를 계산
# transform() 메소드의 결과는 희소 행렬(sparse matrix) 형태로 반환됨
#  결과 배열에서 각 행은 입력된 text_example의 한 문서를 대표하고, 각 열은 CountVectorizer에 의해 학습된 단어의 빈도를 나타냄.
countvect_example.transform(text_example).toarray()    

array([[0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

In [40]:
# CountVectorizer에 의해 구축된 어휘 사전을 나타냄
# 이 사전은 각 단어와 해당 단어에 할당된 고유한 정수 인덱스를 포함하는 딕셔너리 형태
countvect_example.vocabulary_

{'great': 6,
 'game': 5,
 'the': 10,
 'election': 3,
 'was': 12,
 'over': 9,
 'very': 11,
 'clean': 1,
 'match': 8,
 'but': 0,
 'forgettable': 4,
 'it': 7,
 'close': 2}

In [ ]:
# 2.1 베르누이 나이브 베이지안 분류기 / 2.2 다항 나이브 베이지안 분류기 / 2.3 가우시안 나이브 베이지안 분류기
# ------------------------------------------------------------------------------------------------------------------------------------------ 2.1
# 베르누이 나이브 베이지안 분류기(BernoulliNB) : 다루고자 하는 모든 데이터가 불린 피쳐
# • 사용되는 데이터 타입은 이산형 데이터인데, 이러한 데이터를 모두 불린 타입으로 변경하여 학습
# ‐ 정수 타입 숫자라면 임계값 기준으로 True 또는 False로 변환
# ------------------------------------------------------------------------------------------------------------------------------------------ 2.2 
# 다항 나이브 베이지안 분류기(MultinomialNB) : 베르누이 분류기와 달리 각 피쳐들이 이산형이지만, 이진값이 아닌 여러 개의 값을 가질 수 있다
# ------------------------------------------------------------------------------------------------------------------------------------------ 2.3 
#  가우시안 나이브 베이지안 분류기(GaussianNB) : 연속형 값을 피쳐로 가진 데이터의 확률을 구하기 위해 y의 분포를 정규분포(gaussian)로 가정
# • 확률밀도 함수 상의 해당 값 x가 나올 확률로 나이브 베이지안(NB)을 구현
# • 가능도

In [41]:
# 2.1 
from sklearn.naive_bayes import BernoulliNB

# fit 메소드를 사용해 모델을 학습
# X_example는 특성 데이터(여기서는 텍스트의 벡터화된 형태), y_example은 해당 텍스트의 레이블(스포츠: 1, 비스포츠: 0)
clf = BernoulliNB(alpha=1, binarize=0)  # 모델은 텍스트 데이터 처리에 적합하며, 각 단어의 존재 여부(0 또는 1)만을 고려 / alpha=1은 각 카테고리의 각 특성에 최소한 한 번은 발생한다고 가정하는 것을 의미 / 
# binarize=0: 이 매개변수는 입력된 특성 데이터를 이진 값으로 변환하는 임계값을 설정 / binarize=0은 0을 임계값으로 설정하여, 0 이하의 값은 0으로, 0보다 큰 값은 1로 변환
# 이 설정은 특성 데이터가 연속적인 값을 가지는 경우에 유용하며, 이 값을 기반으로 데이터를 이진 형태로 간주
clf.fit(X_example, y_example) # 학습하기

BernoulliNB(alpha=1, binarize=0)

In [42]:
X_test = ["He won the game", "She upgraded her character"]   # X_test는 분류를 위한 새로운 텍스트 데이터
X_test1 = countvect_example.transform(X_test).toarray()     # transform(X_test).toarray()를 통해 새 텍스트를 이전에 학습된 CountVectorizer를 사용하여 벡터화 / 희소 행렬을 배열로 변환
y_test = clf.predict(X_test1)  # 변환된 테스트 데이터에 대해 학습된 모델을 사용하여 레이블을 예측
y_test  #  결과 배열에서 각 값은 해당 문장이 '스포츠' 관련인 경우 1, 그렇지 않은 경우 0으로 표시

# "He won the game" 문장이 스포츠 관련 내용을 명확하게 담고 있어 1로 분류되는 것은 적절해 보입니다. 
# 하지만 "She upgraded her character"은 비디오 게임이나 다른 비스포츠 관련 활동을 암시할 수 있는데, 이 문장 역시 스포츠로 분류되었다는 점에서 모델의 오분류를 보여줄 수 있습니다. 

array([1, 1])

In [43]:
# 2.2
# 데이터 준비 
y_example_text = ["c","c","c", "j"]
text_example = ["Chinese Beijing Chinese", "ChineseChinese Shanghai", "Chinese Macao", "Tokyo JapanChinese"]
countvect_example = CountVectorizer()    # 이 클래스는 텍스트를 토큰화하고 각 토큰의 발생 빈도를 카운트하여 BOW(Bag of Words) 모델을 생성
X_example =countvect_example.fit_transform(text_example)  # 메소드는 입력된 텍스트 데이터에 대해 어휘 사전을 구축하고 각 문서를 벡터 형태로 변환 / 결과는 희소 행렬 형태로 반환
# transform() 메소드를 사용하여 같은 데이터를 다시 벡터화하고 toarray()를 호출하여 희소 행렬을 넘파이 배열로 변환
# 이 배열은 각 문서의 단어 빈도를 나타내며, 각 행은 입력된 text_example의 한 문서를 대표하고 각 열은 어휘 사전의 단어를 나타냄.
countvect_example.transform(text_example).toarray()

array([[1, 2, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 1]])

In [44]:
# vocabulary_ 속성은 CountVectorizer에 의해 구축된 어휘 사전 / 이 사전은 각 단어와 해당 단어에 할당된 고유 인덱스를 포함하는 딕셔너리
countvect_example.vocabulary

In [45]:
# 학습하기 
from sklearn.naive_bayes import MultinomialNB   # 다항 나이브 베이즈 분류기로, 특히 단어 빈도를 특성으로 사용하는 텍스트 분류에 적합합니다.

clf = MultinomialNB(alpha=1)    # alpha=1은 라플라스 스무딩을 적용하는 것 / 이는 모든 카테고리에서 각 단어가 최소 한 번은 발생한 것으로 가정하며, 이는 특성의 확률이 0이 되는 것을 방지

clf.fit(X_example, y_example_text)  # fit() 메소드는 모델을 훈련 데이터에 적합하게 한다. 여기서 X_example은 특성 데이터(벡터화된 텍스트), y_example_text는 각 텍스트에 대한 레이블

MultinomialNB(alpha=1)

In [46]:
# 테스트 데이터 준비 

X_test = ["Chinese Chinese Chinese Tokyo Japan"]
X_test1 = countvect_example.transform(X_test).toarray()
X_test1

array([[0, 3, 0, 0, 0, 0, 1]])

In [47]:
# 예측하기 
y_test = clf.predict(X_test1)      # clf각 입력 샘플에 대해 가장 가능성이 높은 레이블을 반환

# X_test1은 벡터화된 테스트 데이터를 나타냄. 이 데이터는 앞서 CountVectorizer를 사용하여 토큰화 및 벡터화된 텍스트 데이터로, 모델이 학습 과정에서 사용한 같은 어휘 사전을 바탕으로 처리
y_test

# transform() 메소드는 사전에 학습된 CountVectorizer 모델을 사용하여 X_test를 벡터화
# toarray()는 결과를 넘파이 배열로 변환
# 벡터화된 데이터 X_test1는 희소 행렬(sparse matrix) 형태를 배열로 변환하여 각 단어의 빈도를 나타내는 벡터

array(['c'], dtype='<U1')

In [48]:
# 2.3
# 아이리스 데이터 준비
from sklearn import datasets #내장 데이터셋 사용
from sklearn.naive_bayes import GaussianNB
import pandas as pd

iris = datasets.load_iris() #샘플 데이터 load
df_X=pd.DataFrame(iris.data) # feature   iris.data는 붓꽃 데이터의 특성을 포함하고 있는 배열
df_Y=pd.DataFrame(iris.target) #label(0,1,2로 표현)     iris.target은 각 붓꽃 샘플에 대한 레이블(분류 결과)을 담고 있는 배열
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [51]:
# 데이터 분리 
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33)

In [52]:
# 학습하기 
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()   # 모델은 주로 연속적인 데이터를 처리할 때 사용됩니다.
clf.fit(X_train, y_train)
# X_train: 훈련 데이터 세트로, 모델이 학습할 입력 특성(독립 변수)을 포함 / 이 데이터는 모델이 패턴을 학습하는 데 사용
# y_train: 훈련 데이터에 대한 레이블(종속 변수)로, 각 입력 샘플의 클래스 레이블을 포함.

/opt/anaconda3/envs/24ML/lib/python3.11/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [53]:
# 예측하기 
y_pred = clf.predict(X_test)
y_pred

array([0, 0, 1, 0, 1, 0, 2, 1, 1, 1, 1, 2, 2, 0, 0, 2, 0, 1, 2, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 2, 0, 2, 1, 2, 0, 0, 0, 1, 1, 1, 2, 0, 0, 1,
       0, 2, 0, 0, 0, 1])

In [54]:
# 평가하기 
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[19,  0,  0],
       [ 0, 19,  1],
       [ 0,  2,  9]])